📘 1️⃣ Project Introduction

# ⚡ OmniAI Cloud — Unified Vision + NLP + OCR + Translation Suite

This notebook builds a complete multi-module AI platform containing:

- 🖼️ Image Captioning (BLIP Transformer)
- 🔎 OCR with Currency Correction (EasyOCR)
- 🧠 Text Analytics (Sentiment + Keyword Extraction)
- 🌍 Neural Machine Translation (OPUS-MT Models)
- 🎨 Neon Glassmorphism UI with modular pages
- 🚀 Flask backend + ngrok deployment

All tasks are combined into a single cloud dashboard with modern UI, 
lazy-loaded models, Hugging Face integration, and GPU acceleration.


📘 2️⃣ Concepts Covered in This Project

OmniAI Cloud teaches the foundations of multi-modal AI:

1. Vision (BLIP)
   - Image → Text caption generation
   - Transformer-based encoder–decoder models

2. OCR (Optical Character Recognition)
   - EasyOCR GPU pipeline
   - Post-processing for INR “₹” cleanup
   - Automatic formatting

3. NLP Analytics
   - Sentiment classification using DistilBERT
   - Keyword extraction using YAKE
   - Ranking, scoring, text cleanup

4. Translation
   - Traditional encoder–decoder MT using OPUS-MT
   - Language pair mapping logic
   - Caching for fast multi-request translation

5. Web App Architecture
   - Flask routing structure
   - Flash messages
   - Unified base.html layout
   - Static asset management

6. UI/UX Engineering
   - Neon gradient backgrounds
   - Glass panels
   - Responsive grid layout
   - Toast notifications


📘 3️⃣ Install Dependencies

This cell installs all required libraries:

- Flask → backend framework
- transformers → BLIP, BERT, OPUS-MT pipelines
- easyocr → OCR engine with multilingual support
- yake → keyword extraction
- pillow → image processing
- opencv-python-headless → image manipulation without GUI
- torch + torchvision → GPU inference
- pyngrok → deploy the app publicly

Also creates:
templates/
static/
uploads/


In [ ]:
# ===============================
# 1️⃣ Install dependencies
# ===============================
!pip install -q flask transformers easyocr yake pillow opencv-python-headless torch torchvision --upgrade
!mkdir -p templates static uploads

In [ ]:
!pip install pyngrok

📘 4️⃣ Hugging Face Authentication

Your original token has been removed for security.

To load BLIP and OPUS-MT models:

1. Open: https://huggingface.co/settings/tokens  
2. Generate a token → set permission to "Read"
3. Copy the token
4. Replace inside the notebook:

login(token="YOUR_HF_TOKEN_HERE")

⚠️ Never expose your token in GitHub, Kaggle public, or screenshots.


In [ ]:
# ===============================
# 2️⃣ Hugging Face Authentication
# ===============================
from huggingface_hub import login
login(token="YOUR_HF_TOKEN_HERE")  # 🔑 replace with your HF token


📘 5️⃣ Application Architecture Overview

OmniAI Cloud is structured as a modular AI suite:

          ┌──────────────┐
          │   Home UI    │
          └──────┬───────┘
                 │
     ┌───────────┼────────────┬─────────────┬──────────────┐
     ▼           ▼            ▼             ▼               ▼
Image Caption  OCR        Text Analytics  Translation     About
(BLIP)         (EasyOCR)  (Sentiment +    (OPUS-MT)
                            YAKE)

Each module has:
- Its own Flask route
- Its own template
- Lazy-loaded model for performance
- Beautiful neon–glass UI styling

Shared:
- base.html (layout)
- style.css (global theme)
- Unified flash notifications
- Single uploads directory


📘 6️⃣ Image Captioning (BLIP)

Uses:
  Salesforce/blip-image-captioning-base

Flow:
1. Upload image
2. Validate format + size
3. Preprocess using BLIP processor
4. Generate caption with BLIP model.generate()
5. Display result inside glowing caption bubble

Supports:
- PNG / JPG / JPEG / WEBP up to 20MB


📘 7️⃣ OCR with Currency Correction

OCR Engine:
- EasyOCR (GPU enabled if available)

Pipeline:
1. Read and preprocess image
2. Extract full paragraphs (detail=0, paragraph=True)
3. Cleanup + normalization via custom regex rules:
   - Detect prices
   - Convert "INR", "Rs", or text-detected "7xxxx" → ₹xxxx

Displayed using:
<pre class="mono">…</pre>

This ensures neat, readable OCR output suitable for invoices and receipts.


📘 8️⃣ Text Analytics

Two NLP modules are provided:

1. Sentiment Analysis:
   Model: distilbert-base-uncased-finetuned-sst-2-english
   Output:
     - Label (POSITIVE/NEGATIVE)
     - Confidence score (0–100%)

2. Keyword Extraction:
   Using YAKE:
     - Top 10 ranked keywords
     - Deduplication (dedupLim=0.9)
     - Score-based sorting

Results displayed in:
- Chips (sentiment)
- Table-style grid (keywords)


📘 9️⃣ Neural Machine Translation

Model: Helsinki-NLP/opus-mt-<src>-<tgt>

Pipeline:
1. User selects Source and Target languages
2. Text is validated
3. Translation pipeline loads via @lru_cache
4. Output is wrapped in glowing caption bubble

Supported languages:
- English
- French
- German
- Spanish
- Hindi
- Italian
- Chinese
- Japanese
- Russian


📘 🔟 Flask Routing Overview

Routes included:

GET /
 → Dashboard home

GET /vision
POST /vision
 → BLIP Image Captioning

GET /ocr
POST /ocr
 → EasyOCR extraction

GET /nlp
POST /nlp
 → Sentiment + Keyword extraction

GET /translate
POST /translate
 → Neural machine translation

GET /about
 → Project details

GET /uploads/<file>
 → Serve uploaded images back to UI


In [ ]:
# ===============================
# 3️⃣ Create app.py (Main Flask Application)
# ===============================
%%writefile app.py
import os, functools, uuid, torch, re
from flask import Flask, render_template, request, redirect, url_for, send_from_directory, flash
from werkzeug.utils import secure_filename
from transformers import BlipProcessor, BlipForConditionalGeneration, pipeline
import easyocr
from PIL import Image
import numpy as np
import yake

# -------------------------------
# Flask setup
# -------------------------------
app = Flask(__name__)
app.secret_key = "omni_secret_" + str(uuid.uuid4())[:8]
app.config["UPLOAD_FOLDER"] = "uploads"
app.config["MAX_CONTENT_LENGTH"] = 20 * 1024 * 1024  # 20 MB

ALLOWED_IMG = {"png", "jpg", "jpeg", "webp"}

def allowed_file(filename, exts):
    return "." in filename and filename.rsplit(".", 1)[1].lower() in exts

# -------------------------------
# Lazy loaders for models
# -------------------------------
@functools.lru_cache(maxsize=1)
def get_image_captioner():
    """BLIP model for image captioning"""
    processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
    model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")
    return processor, model

@functools.lru_cache(maxsize=1)
def get_sentiment():
    return pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

@functools.lru_cache(maxsize=1)
def get_keyword_extractor():
    return yake.KeywordExtractor(lan="en", n=1, top=10, dedupLim=0.9)

@functools.lru_cache(maxsize=1)
def get_ocr_reader():
    return easyocr.Reader(["en"], gpu=torch.cuda.is_available())

# -------------------------------
# OCR Post-Processing Correction
# -------------------------------
def clean_ocr_text(text):
    text = re.sub(r"(?<=\b)7(?=\s*\d{1,3}(?:,?\d{3})*)", "₹", text)
    text = re.sub(r"^7(?=\d{1,3}(?:,?\d{3})*)", "₹", text)
    text = re.sub(r"(?<=\s)7(?=\d{1,3}(?:,?\d{3})*)", "₹", text)
    text = re.sub(r"\bINR\s*", "₹", text, flags=re.IGNORECASE)
    text = re.sub(r"Rs\.?\s*", "₹", text, flags=re.IGNORECASE)
    text = re.sub(r"\s{2,}", " ", text)
    return text.strip()

# -------------------------------
# Routes
# -------------------------------
@app.route("/")
def home():
    return render_template("index.html")

@app.route("/about")
def about():
    return render_template("about.html")

# ---------- Vision (BLIP Captioning) ----------
@app.route("/vision", methods=["GET", "POST"])
def vision():
    caption = None
    filename = None
    if request.method == "POST":
        file = request.files.get("image")
        if not file or file.filename == "":
            flash("Please upload an image.", "warn")
            return redirect(url_for("vision"))
        if not allowed_file(file.filename, ALLOWED_IMG):
            flash("Unsupported file type. Use png/jpg/jpeg/webp.", "error")
            return redirect(url_for("vision"))

        filename = secure_filename(file.filename)
        save_path = os.path.join(app.config["UPLOAD_FOLDER"], f"{uuid.uuid4().hex}_{filename}")
        file.save(save_path)

        try:
            processor, model = get_image_captioner()
            image = Image.open(save_path).convert("RGB")
            inputs = processor(image, return_tensors="pt").to(model.device)
            out = model.generate(**inputs)
            caption = processor.decode(out[0], skip_special_tokens=True)
        except Exception as e:
            flash(f"Image captioning failed: {e}", "error")
            caption = None

        filename = os.path.basename(save_path)
    return render_template("image_classification.html", caption=caption, filename=filename)

# ---------- OCR ----------
@app.route("/ocr", methods=["GET", "POST"])
def ocr():
    text = None
    filename = None
    if request.method == "POST":
        file = request.files.get("image")
        if not file or file.filename == "":
            flash("Please upload an image.", "warn")
            return redirect(url_for("ocr"))
        if not allowed_file(file.filename, ALLOWED_IMG):
            flash("Unsupported file type. Use png/jpg/jpeg/webp.", "error")
            return redirect(url_for("ocr"))

        filename = secure_filename(file.filename)
        save_path = os.path.join(app.config["UPLOAD_FOLDER"], f"{uuid.uuid4().hex}_{filename}")
        file.save(save_path)

        reader = get_ocr_reader()
        try:
            img = Image.open(save_path).convert("RGB")
            arr = np.array(img)
            ocr_out = reader.readtext(arr, detail=0, paragraph=True)
            raw_text = "\n".join([t.strip() for t in ocr_out if t and t.strip()])
            text = clean_ocr_text(raw_text)
        except Exception as e:
            flash(f"OCR failed: {e}", "error")
            text = None

        filename = os.path.basename(save_path)
    return render_template("ocr.html", text=text, filename=filename)

# ---------- Text Analytics ----------
@app.route("/nlp", methods=["GET", "POST"])
def nlp():
    sentiment = None
    keywords = []
    user_text = ""
    if request.method == "POST":
        user_text = request.form.get("text", "").strip()
        if not user_text:
            flash("Please enter some text.", "warn")
            return redirect(url_for("nlp"))
        try:
            sent_pipe = get_sentiment()
            sentiment = sent_pipe(user_text)[0]
        except Exception as e:
            flash(f"Sentiment failed: {e}", "error")
        try:
            ke = get_keyword_extractor()
            kws = ke.extract_keywords(user_text)
            kws = sorted(kws, key=lambda x: x[1])[:10]
            keywords = [{"term": k, "score": round(s, 4)} for k, s in kws]
        except Exception as e:
            flash(f"Keyword extraction failed: {e}", "error")
    return render_template("text_analytics.html", sentiment=sentiment, keywords=keywords, user_text=user_text)

# ---------- Translation ----------
@functools.lru_cache(maxsize=1)
def get_translator(src="en", tgt="fr"):
    model_name = f"Helsinki-NLP/opus-mt-{src}-{tgt}"
    return pipeline("translation", model=model_name)

LANG_MAP = {
    "en": "English",
    "fr": "French",
    "de": "German",
    "es": "Spanish",
    "hi": "Hindi",
    "it": "Italian",
    "zh": "Chinese",
    "ja": "Japanese",
    "ru": "Russian"
}

@app.route("/translate", methods=["GET", "POST"])
def translate():
    output_text = None
    src_lang = "en"
    tgt_lang = "fr"
    input_text = ""
    if request.method == "POST":
        input_text = request.form.get("text", "").strip()
        src_lang = request.form.get("source_lang", "en")
        tgt_lang = request.form.get("target_lang", "fr")
        if not input_text:
            flash("Please enter text to translate.", "warn")
            return redirect(url_for("translate"))
        try:
            translator = get_translator(src_lang, tgt_lang)
            translated = translator(input_text, max_length=400)
            output_text = translated[0]["translation_text"]
        except Exception as e:
            flash(f"Translation failed: {e}", "error")
    return render_template("translate.html", LANG_MAP=LANG_MAP, src_lang=src_lang,
                           tgt_lang=tgt_lang, input_text=input_text, output_text=output_text)

# ---------- Serve uploads ----------
@app.route("/uploads/<path:fname>")
def uploads(fname):
    return send_from_directory(app.config["UPLOAD_FOLDER"], fname)

if __name__ == "__main__":
    os.makedirs(app.config["UPLOAD_FOLDER"], exist_ok=True)
    app.run(host="0.0.0.0", port=8000, debug=False)


In [ ]:
%%writefile templates/base.html
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <title>OmniAI Cloud – Mini</title>
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
  <link rel="preconnect" href="https://fonts.googleapis.com">
  <link href="https://fonts.googleapis.com/css2?family=Inter:wght@400;600;800&display=swap" rel="stylesheet">
</head>
<body>
  <div class="bg"></div>

  <nav class="nav">
    <a class="brand" href="{{ url_for('home') }}">⚡ OmniAI Cloud</a>
    <div class="links">
      <a href="{{ url_for('vision') }}">🖼️ Vision</a>
      <a href="{{ url_for('ocr') }}">🔎 OCR</a>
      <a href="{{ url_for('nlp') }}">🧠 Text Analytics</a>
      <a href="{{ url_for('translate') }}">🌍 Translate</a>
      <a href="{{ url_for('about') }}">ℹ️ About</a>
    </div>
  </nav>

  <main class="container">
    {% with messages = get_flashed_messages(with_categories=true) %}
      {% if messages %}
      <div class="toast-wrap">
        {% for cat, msg in messages %}
          <div class="toast {{ cat }}">{{ msg }}</div>
        {% endfor %}
      </div>
      {% endif %}
    {% endwith %}
    {% block content %}{% endblock %}
  </main>

  <footer class="footer">
    <span>© {{ 2025 }} OmniAI Cloud • Neon UI</span>
  </footer>
</body>
</html>


📘 1️⃣1️⃣ Frontend Templates

Templates included:

base.html  
- Shared layout
- Navbar + footer
- Toast messages
- Background neon gradients

index.html  
- Hero section
- Feature grid

image_classification.html  
- Upload → Caption pipeline
- BLIP results bubble

ocr.html  
- Preview uploaded image
- Extracted text area

text_analytics.html  
- Sentiment + keywords table

translate.html  
- Language dropdowns
- Glowing translation output

about.html  
- Tech stack showcase
- Description of each module


In [ ]:
%%writefile templates/index.html
{% extends "base.html" %}
{% block content %}
<section class="hero glass glow">
  <h1>Unified Vision & Language Intelligence</h1>
  <p>Run image captioning, OCR, text analytics, and translation in a single futuristic dashboard.</p>
  <div class="cta-row">
    <a class="btn primary" href="{{ url_for('vision') }}">🖼️ Try Image Captioning</a>
    <a class="btn" href="{{ url_for('ocr') }}">🔎 Run OCR</a>
    <a class="btn" href="{{ url_for('nlp') }}">🧠 Analyze Text</a>
    <a class="btn" href="{{ url_for('translate') }}">🌍 Translate Text</a>
  </div>
</section>

<section class="grid">
  <div class="card glass hover-rise">
    <h3>🖼️ Vision</h3>
    <p>Generate descriptive captions for uploaded images.</p>
    <a class="link" href="{{ url_for('vision') }}">Open</a>
  </div>
  <div class="card glass hover-rise">
    <h3>🔎 OCR</h3>
    <p>Extract text from images and scanned documents.</p>
    <a class="link" href="{{ url_for('ocr') }}">Open</a>
  </div>
  <div class="card glass hover-rise">
    <h3>🧠 Text Analytics</h3>
    <p>Analyze sentiment and extract key terms from text.</p>
    <a class="link" href="{{ url_for('nlp') }}">Open</a>
  </div>
  <div class="card glass hover-rise">
    <h3>🌍 Translation</h3>
    <p>Translate text seamlessly across multiple languages.</p>
    <a class="link" href="{{ url_for('translate') }}">Open</a>
  </div>
</section>
{% endblock %}


In [ ]:
%%writefile templates/image_classification.html
{% extends "base.html" %}
{% block content %}
<div class="page-title">
  <h2>🪄 Image Captioning</h2>
  <p>Upload an image to generate a smart descriptive caption using BLIP.</p>
</div>

<form class="uploader glass" method="post" enctype="multipart/form-data">
  <label for="image" class="dropzone">
    <span>Drag & drop or click to upload</span>
    <small>PNG / JPG / JPEG / WEBP • max 20MB</small>
  </label>
  <input id="image" name="image" type="file" accept="image/*" required />
  <button class="btn primary" type="submit">Generate Caption 🚀</button>
</form>

{% if filename %}
<div class="preview-wrap">
  <img class="preview" src="{{ url_for('uploads', fname=filename) }}" alt="uploaded">
</div>
{% endif %}

{% if caption %}
<div class="results glass glow">
  <h3>Generated Caption ✨</h3>
  <div class="caption-bubble">
    <p class="caption-text">"{{ caption }}"</p>
  </div>
</div>
{% endif %}

<style>
.caption-bubble {
  margin-top: 15px;
  padding: 18px 22px;
  border-radius: 14px;
  border: 1px solid rgba(45, 226, 230, 0.4);
  background: linear-gradient(145deg, rgba(45, 226, 230, 0.1), rgba(154, 77, 255, 0.1));
  box-shadow: 0 0 20px rgba(45, 226, 230, 0.2), 0 0 40px rgba(154, 77, 255, 0.15);
  text-align: center;
  animation: glowPulse 3s ease-in-out infinite;
}

.caption-text {
  font-size: 1.25rem;
  color: #e4e8ff;
  font-weight: 500;
  letter-spacing: 0.4px;
  text-shadow: 0 0 10px rgba(45, 226, 230, 0.4);
}

@keyframes glowPulse {
  0% { box-shadow: 0 0 15px rgba(45, 226, 230, 0.2); }
  50% { box-shadow: 0 0 30px rgba(154, 77, 255, 0.35); }
  100% { box-shadow: 0 0 15px rgba(45, 226, 230, 0.2); }
}
</style>
{% endblock %}


In [ ]:
%%writefile templates/ocr.html
{% extends "base.html" %}
{% block content %}
<div class="page-title">
  <h2>🔎 OCR – Text Detection</h2>
  <p>Extract text from uploaded images and scanned documents with smart currency correction.</p>
</div>

<form class="uploader glass" method="post" enctype="multipart/form-data" id="ocrForm">
  <label for="image" class="dropzone">
    <span>Drag & drop or click to upload</span>
    <small>PNG / JPG / JPEG / WEBP • max 20MB</small>
  </label>
  <input id="image" name="image" type="file" accept="image/*" required onchange="previewImage(event)" />
  <button class="btn primary" type="submit">Run OCR ✨</button>
</form>

<div class="preview-wrap" id="previewWrap" style="display:none;">
  <h3 style="margin-bottom:8px;">📷 Uploaded Image</h3>
  <img id="previewImg" class="preview" src="#" alt="uploaded preview">
</div>

{% if text %}
<div class="results glass glow">
  <h3>🧾 Extracted Text</h3>
  <pre class="mono">{{ text }}</pre>
</div>
{% endif %}

<script>
function previewImage(event) {
  const input = event.target;
  const previewWrap = document.getElementById('previewWrap');
  const previewImg = document.getElementById('previewImg');
  if (input.files && input.files[0]) {
    const reader = new FileReader();
    reader.onload = function(e) {
      previewImg.src = e.target.result;
      previewWrap.style.display = "block";
    };
    reader.readAsDataURL(input.files[0]);
  }
}
</script>
{% endblock %}


In [ ]:
with open("templates/text_analytics.html", "w") as f:
    f.write("""{% extends "base.html" %}
{% block content %}
<div class="page-title">
  <h2>🧠 Text Analytics</h2>
  <p>Sentiment classification + keyword extraction.</p>
</div>

<form class="card glass" method="post">
  <textarea name="text" rows="7" placeholder="Paste or type text here..." required>{{ user_text }}</textarea>
  <button class="btn primary" type="submit">Analyze 🔬</button>
</form>

{% if sentiment %}
<div class="results glass glow">
  <h3>Sentiment</h3>
  <div class="chips">
    <span class="chip">Label: {{ sentiment['label'] }}</span>
    <span class="chip">Confidence: {{ (sentiment['score'] * 100) | round(2) }}%</span>
  </div>
</div>
{% endif %}

{% if keywords %}
<div class="results glass">
  <h3>Keywords</h3>
  <div class="table">
    <div class="row header"><div>Keyword</div><div>Score</div></div>
    {% for kw in keywords %}
      <div class="row"><div>{{ kw.term }}</div><div>{{ kw.score }}</div></div>
    {% endfor %}
  </div>
</div>
{% endif %}
{% endblock %}""")


In [ ]:
%%writefile templates/translate.html
{% extends "base.html" %}
{% block content %}
<div class="page-title">
  <h2>🌍 Text Translation</h2>
  <p>Translate text between multiple languages using neural machine translation models.</p>
</div>

<form class="card glass" method="post">
  <div style="display:flex; gap:10px; flex-wrap:wrap;">
    <div style="flex:1;">
      <label>From:</label>
      <select name="source_lang" class="lang-select">
        {% for code, name in LANG_MAP.items() %}
          <option value="{{ code }}" {% if code == src_lang %}selected{% endif %}>{{ name }}</option>
        {% endfor %}
      </select>
    </div>
    <div style="flex:1;">
      <label>To:</label>
      <select name="target_lang" class="lang-select">
        {% for code, name in LANG_MAP.items() %}
          <option value="{{ code }}" {% if code == tgt_lang %}selected{% endif %}>{{ name }}</option>
        {% endfor %}
      </select>
    </div>
  </div>

  <textarea name="text" rows="6" placeholder="Enter text to translate..." required>{{ input_text }}</textarea>
  <button class="btn primary" type="submit">Translate 🔁</button>
</form>

{% if output_text %}
<div class="results glass glow">
  <h3>Translated Output ✨</h3>
  <div class="caption-bubble">
    <p class="caption-text">{{ output_text }}</p>
  </div>
</div>
{% endif %}

<style>
.lang-select {
  width: 100%;
  padding: 10px;
  border-radius: 10px;
  border: 1px solid rgba(255,255,255,0.2);
  background: rgba(255,255,255,0.08);
  color: #fff;
  font-size: 0.95rem;
}
.lang-select option {
  color: black;
}
.caption-bubble {
  margin-top: 15px;
  padding: 18px 22px;
  border-radius: 14px;
  border: 1px solid rgba(45, 226, 230, 0.4);
  background: linear-gradient(145deg, rgba(45, 226, 230, 0.1), rgba(154, 77, 255, 0.1));
  box-shadow: 0 0 20px rgba(45, 226, 230, 0.2), 0 0 40px rgba(154, 77, 255, 0.15);
  text-align: center;
  animation: glowPulse 3s ease-in-out infinite;
}
.caption-text {
  font-size: 1.2rem;
  color: #e4e8ff;
  text-shadow: 0 0 10px rgba(45, 226, 230, 0.4);
}
@keyframes glowPulse {
  0% { box-shadow: 0 0 15px rgba(45, 226, 230, 0.2); }
  50% { box-shadow: 0 0 30px rgba(154, 77, 255, 0.35); }
  100% { box-shadow: 0 0 15px rgba(45, 226, 230, 0.2); }
}
</style>
{% endblock %}


In [ ]:
%%writefile templates/about.html
{% extends "base.html" %}
{% block content %}
<div class="page-title">
  <h2>ℹ️ About OmniAI Cloud (Mini)</h2>
  <p>An integrated AI platform bringing Vision, OCR, Text Analytics, and Translation into one futuristic dashboard.</p>
</div>

<div class="card glass">
  <ul class="bullets">
    <li>🖼️ Image Captioning • <em>BLIP (Salesforce/blip-image-captioning-base)</em></li>
    <li>🔎 OCR • <em>EasyOCR (GPU-accelerated text extraction with currency correction)</em></li>
    <li>🧠 Text Analytics • <em>Sentiment (DistilBERT) + Keywords (YAKE)</em></li>
    <li>🌍 Text Translation • <em>Helsinki-NLP/opus-mt multilingual transformer models</em></li>
  </ul>

  <p>OmniAI Cloud (Mini) is built for Google Colab, featuring real-time GPU inference,
  Hugging Face integration, and a glowing neon UI powered by glassmorphism for a modern AI-as-a-Service experience.</p>

  <p style="margin-top:10px; color:#b7c0d8;">
    All modules share a unified interface, interactive previews, and a consistent visual style —
    making it a perfect demo for end-to-end AI applications on the cloud.
  </p>
</div>
{% endblock %}


📘 1️⃣2️⃣ Neon Glassmorphism UI (CSS)

The UI includes:

- Multi-gradient neon backgrounds
- Smooth glass cards (backdrop-filter blur)
- Glowing edges on hover
- Animated CTA buttons
- Responsive grid system
- Toast notifications for warnings/errors
- Monospace formatted OCR text
- Adaptive layout for mobile devices


In [ ]:
%%writefile static/style.css
:root{
  --bg1:#0e0f24;
  --bg2:#111b4b;
  --cyan:#2de2e6;
  --violet:#9a4dff;
  --blue:#3b82f6;
  --white:#f5f7ff;
  --muted:#b7c0d8;
  --glass: rgba(255,255,255,0.08);
  --border: rgba(255,255,255,0.16);
  --shadow: 0 10px 30px rgba(0,0,0,0.35);
}

*{box-sizing:border-box}
html,body{height:100%}
body{
  margin:0; font-family:Inter,system-ui,-apple-system,Segoe UI,Roboto,Arial;
  color:var(--white); background:var(--bg1); overflow-x:hidden;
}

.bg{
  position:fixed; inset:0; z-index:-1;
  background: radial-gradient(1000px 800px at 20% 10%, rgba(45,226,230,0.25), transparent 60%),
              radial-gradient(900px 700px at 80% 30%, rgba(154,77,255,0.25), transparent 60%),
              radial-gradient(700px 600px at 50% 80%, rgba(59,130,246,0.25), transparent 60%),
              linear-gradient(140deg, var(--bg1), var(--bg2));
  filter:saturate(120%);
}

.nav{
  display:flex; align-items:center; justify-content:space-between;
  padding:14px 22px; position:sticky; top:0; backdrop-filter: blur(8px);
  background:linear-gradient(to right, rgba(17,17,35,0.7), rgba(17,27,75,0.4));
  border-bottom:1px solid var(--border); z-index:10;
}
.brand{font-weight:800; text-decoration:none; color:var(--white); letter-spacing:.2px}
.links a{margin-left:14px; text-decoration:none; color:var(--muted)}
.links a:hover{color:var(--white)}

.container{max-width:1024px; margin:32px auto; padding:0 18px}

.footer{
  text-align:center; color:var(--muted); padding:24px 0; border-top:1px solid var(--border);
  margin-top:50px; background:linear-gradient(to top, rgba(17,27,75,0.25), transparent);
}

.hero{
  text-align:center; padding:36px 28px; border-radius:18px; box-shadow:var(--shadow);
}
.hero h1{margin:0 0 10px; font-size:36px; letter-spacing:.3px}
.hero p{margin:0 0 18px; color:var(--muted)}
.cta-row{display:flex; gap:12px; justify-content:center; flex-wrap:wrap}

.btn{
  padding:12px 16px; border-radius:12px; border:1px solid var(--border);
  color:var(--white); text-decoration:none; display:inline-flex; align-items:center; gap:8px;
  background:linear-gradient(180deg, rgba(255,255,255,0.05), rgba(255,255,255,0.02));
  transition:transform .15s ease, box-shadow .15s ease, border-color .2s ease;
}
.btn.primary{
  border-color:transparent;
  background:linear-gradient(90deg, var(--cyan), var(--violet), var(--blue));
  background-size:200% 100%; animation:shine 5s linear infinite;
}
.btn:hover{transform:translateY(-1px); box-shadow:0 8px 20px rgba(0,0,0,.35)}
@keyframes shine{0%{background-position:0% 0}100%{background-position:200% 0}}

.grid{
  display:grid; grid-template-columns:repeat(auto-fit, minmax(220px,1fr)); gap:16px; margin-top:22px;
}
.card{
  padding:18px; border-radius:16px; border:1px solid var(--border); background:var(--glass);
}
.link{color:var(--cyan); text-decoration:none}
.link:hover{text-decoration:underline}

.glass{backdrop-filter: blur(10px)}
.glow{box-shadow:0 0 0 1px rgba(45,226,230,.15), 0 0 40px rgba(154,77,255,.15)}

.hover-rise{transition:transform .18s ease, box-shadow .18s ease}
.hover-rise:hover{transform:translateY(-3px); box-shadow:0 16px 40px rgba(0,0,0,.4)}

.page-title h2{margin:.2rem 0 .25rem}
.page-title p{margin:0; color:var(--muted)}

.uploader{
  margin:16px 0 10px; padding:18px; border-radius:16px; background:var(--glass); border:1px solid var(--border);
  display:flex; flex-direction:column; gap:14px;
}
.dropzone{
  display:flex; flex-direction:column; align-items:center; justify-content:center;
  height:160px; border:2px dashed rgba(255,255,255,0.25); border-radius:14px; cursor:pointer;
  color:var(--muted); transition:border-color .15s ease, color .15s ease;
}
.dropzone:hover{border-color:var(--cyan); color:var(--white)}
.uploader input[type=file]{display:none}

.preview-wrap{margin:14px 0}
.preview{
  width:100%; max-height:360px; object-fit:contain; border-radius:12px; border:1px solid var(--border);
  background:rgba(0,0,0,.25)
}

.results{margin:18px 0; padding:18px; border-radius:16px; border:1px solid var(--border); background:var(--glass)}
.chips{display:flex; flex-wrap:wrap; gap:10px; margin-top:6px}
.chip{
  padding:8px 12px; border-radius:999px; border:1px solid var(--border); background:rgba(255,255,255,0.06);
}

.card textarea{
  width:100%; border:none; outline:none; resize:vertical; padding:12px;
  border-radius:12px; color:var(--white); background:rgba(255,255,255,0.06);
}

.table{display:grid; gap:6px; margin-top:8px}
.row{display:grid; grid-template-columns:2fr 1fr; gap:10px; padding:10px; border-radius:10px; background:rgba(255,255,255,0.05)}
.row.header{font-weight:600; background:rgba(255,255,255,0.08)}

.mono{
  white-space:pre-wrap; word-wrap:break-word; font-family:ui-monospace, SFMono-Regular, Menlo, Monaco, Consolas, "Liberation Mono", monospace;
  background:rgba(0,0,0,.25); padding:12px; border-radius:12px; border:1px solid var(--border);
}

/* Toasts */
.toast-wrap{position:fixed; right:16px; bottom:16px; display:flex; flex-direction:column; gap:10px; z-index:20}
.toast{
  padding:10px 12px; border-radius:12px; border:1px solid var(--border); background:rgba(20,20,35,0.85);
  box-shadow:var(--shadow); animation:fadeIn .3s ease both;
}
.toast.warn{border-color:#f59e0b; color:#fde68a}
.toast.error{border-color:#ef4444; color:#fecaca}
@keyframes fadeIn{from{opacity:0; transform:translateY(6px)} to{opacity:1; transform:translateY(0)}}


📘 1️⃣3️⃣ Kill Previous Flask/ngrok Sessions

Running multiple Flask servers can cause:
- Port 8000 already in use
- ngrok tunnel conflicts

Solution:
!pkill -f flask
!pkill -f ngrok

Check running processes:
!lsof -i :8000

Kill specific PID:
!kill -9 <PID>


In [ ]:
# ===============================
# 6️⃣ Kill any previous processes
# ===============================
!pkill -f flask || echo "No flask running"
!pkill -f ngrok || echo "No ngrok running"


In [ ]:
!lsof -i :8000

In [ ]:
!kill -9 6620

📘 1️⃣4️⃣ Run Flask in Background

Start server silently:

!nohup python app.py > flask.log 2>&1 &

This keeps the server running even if the cell is stopped.


In [ ]:
# ===============================
# 7️⃣ Run Flask in the background
# ===============================
!nohup python app.py > flask.log 2>&1 &

📘 1️⃣5️⃣ Start ngrok Tunnel

To expose your app publicly:

1. Create account:
   https://dashboard.ngrok.com/signup

2. Get authtoken:
   https://dashboard.ngrok.com/get-started/your-authtoken

3. Insert token:
   conf.get_default().auth_token = "YOUR_NGROK_TOKEN"

4. Create tunnel:
   public_url = ngrok.connect(8000)

Share this public URL with anyone — the full app runs online.


In [ ]:
# ===============================
# 8️⃣ Start ngrok tunnel
# ===============================
from pyngrok import ngrok, conf
conf.get_default().auth_token = "YOUR_NGROK_TOKEN"  # 🔑 replace with your token

public_url = ngrok.connect(8000)
print("🌍 Public URL:", public_url)

# ===============================
# 9️⃣ Check logs (optional)
# ===============================
!sleep 3 && tail -n 20 flask.log


📘 1️⃣6️⃣ View Logs

Check the last 20–50 lines:

!tail -n 20 flask.log
!tail -n 50 flask.log

Useful for:
- BLIP model errors
- OCR reader failures
- Translation fallback issues
- Upload path mistakes


In [ ]:
!tail -n 50 flask.log

In [ ]:
📘 1️⃣7️⃣ OmniAI Cloud — Completed

🎉 Your OmniAI Cloud platform is ready!

Users can:

✔ Caption images  
✔ Extract OCR text with currency correction  
✔ Analyze sentiment and keywords  
✔ Translate text across 9 languages  
✔ Enjoy a neon futuristic UI  
✔ Access everything through ngrok links  

A perfect end-to-end cloud AI dashboard for demos, portfolio, or teaching.
